# Introduction

This article is part of the final homework for Data Anlysis & Geometry. The article (I think) is also designed to answer the questions that audience have due to shortage of the speech time. The article is not intended to be a comprehensive review of the fields of Generative model. Many excellent papers and algorithm is ignored in the article due to the limitation of the length of an article.The generative model is really an interesting reserach aspect.


 The article is intended to describes:
 three parts of generative models are introduced in this article：
 1.  Two important models in generative models fields, they are: VAE(variational Autoencoder), vanilla GAN(generative adversarial network)
 2.  Some critical mathematical inductions to reveal the key ideas of them
 3.  Some advantages & disadvantages they have respectively
 4.  Experimental results in mnist character generation


# General Principle
## Maximum Likelihood

These three models, they all work via the principle of maximum likelihood. But VAE work with MLE directly and GAN series work on it indirectly(by sampling from $p_{data}$), by focusing on maximum likelihood, we can eliminate some of the more distracting differences between these three models

The basic idea of maximum likelihood is to define a model that provides an estimate of a probability distribution, parameterized by parameters $\theta$. We then refer to the likelihood as the probability that the model assigns to the trainingdata:$\prod_{i=1}^{m}p_{model}(x(i);\theta)$,for a dataset containing $m$ training examples.

The principle of maximum likelihood simply says to choose the parametersfor the model that maximize the likelihood of the training data.  This is easiestto do in log space, where we have a sum rather than a product over examples.

<center>$\theta^{*} = \arg\max_\theta \prod_{i=1}^m p_\theta(\mathbf{x}^{(i)})$</center>
<center>$\theta^{*} = \arg\max_\theta \sum_{i=1}^n \log p_\theta(\mathbf{x}^{(i)})$<center>
    
Actually, We can also think of maximum likelihood estimation as minimizing the KL divergence between the data generating distribution and the model
    
<center>$\boldsymbol{\theta}^{*}=\underset{\boldsymbol{\theta}}{\arg \min } D_{\mathrm{KL}}\left(p_{\text {data }}(\boldsymbol{x}) \| p_{\text {model }}(\boldsymbol{x} ; \boldsymbol{\theta})\right)$<center>
    
The Generative model working via maximun likelihood have several variants, judging from the relationship with the data distribution and the tractablity of the density function, which is illustrated in the following picture：

<img src=\"Taxonomy%20of%20Generative%20Models.png" width="450"/>

And we will focus on VAE in Explicit models requiring approximation and GAN series in Implicit density models

## KL divergence & JS divergence

### KL divergence
Kl divergence measure the difference between two distribution, which works as an effective measure for generative model

<center> $D_{KL}(p \mid \mid q) = \int_x p(x)\log p(x) dx - \int_x p(x) \log q(x) dx = \int_x p(x) \log \frac{p(x)}{q(x)} dx$ <center>
    
Obviously，Kl divergence has no symmetry, $d(p,q) = d(q,p)$, thus it can't be used as metric, but its asymmetry also induce some special treatment.
    
 Since $KL(P||Q) \neq KL(Q||P)$, we have Forward KL and Reverse KL, which has zero-avoiding and zero-forcing property respectively. 
 


1. Forward KL 
 
 The penalty function contributes loss to the total KL wherever $p(Z) > 0$ ,For $p(Z)>0$, $\lim_{q(Z) \to 0} \log \frac{p(z)}{q(z)} \to \infty$, This means that the forward-KL will be large wherever $Q(Z)$ fails to "cover up" $P(Z)$ 
 
<center> $\begin{align*} 
KL(P||Q) & = \sum_z p(z) \log \frac{p(z)}{q(z)} \\ 
& = \mathbb{E}_{p(z)}{\big[\log \frac{p(z)}{q(z)}\big]}\\ 
\end{align*}$<center>
    
 <img src=\"forward-KL.png" width="450"/>


2.Reverse KL

<center> $\begin{align*} 
KL(Q||P) & = \sum_z q(z) \log \frac{q(z)}{p(z)} \\ 
& = \mathbb{E}_{p(z)}{\big[\log \frac{q(z)}{p(z)}\big]} 
\end{align*} $<center>
 
 <img src=\"reverse-KL.png" width="450"/>

And KL divergence usually appears in ML(maximum likelihood-guided) Learning, which does is to maximize the likelihood of making those real data observations given that set of parameters.And we make the notation:
* $P_\theta(x)$ as q(x)
* Empirical distribution $\hat{P}_D(x) = \sum_{i=1}^N \frac{1}{N}\delta_(x-x_i)$ as p(x)

then we can state that:

<center>$D_{KL}(\hat{P}_D(x) \mid \mid P_\theta(x)) = \int_x \hat{P}_D(x) \log \frac{\hat{P}_D(x)}{P_\theta(x)} dx = -H[\hat{P}_D(x)] - \int_x \hat{P}_D(x) \log P_\theta(x) dx$<center>

Compute the gradient of the divergence, 

$\nabla_\theta(D_{KL}(\hat{P}_D(x) \mid \mid P_\theta(x))) = \nabla_\theta \mathbb{E}_{\hat{P}_D(x)} \log P_\theta(x)$ 

take distribution into expectation,

$\nabla_\theta \sum_x \hat{P}_D(x) \log P_\theta(x) = \nabla_\theta \sum_x \left[ \frac{1}{N} \sum_{i=1}^N \delta_(x-x_i)   \right] \log P(x \mid \theta)  = \frac{1}{N} \sum_x \nabla_\theta \log P(x \mid \theta)$

### JS divergence
Some people always contribute gan's success partially to JS divergence, which have symmetric, avoiding the problem of specific fit that KL divergence have

JS divergence:
$ D_{JS}(p \| q) = \frac{1}{2} D_{KL}(p \| \frac{p + q}{2}) + \frac{1}{2} D_{KL}(q \| \frac{p + q}{2}) $

## Normal distribution for Prior

To avoid the complicated construction process of latent variables, such as deciding by hand what information each dimension of latent variable encodes, sometimes an unusual shortcut will be taken to dealing with the problem:

Assume that there is no simple interpretation of the dimensions of latent variable , and instead assert that samples of latent variable can be drawn from a simple distribution, like $N(0,I)$

The key is to notice that any distribution in d dimensions can be generated by taking a set of d variables that are normally distributed and mapping them through a sufficiently complicated function. For example,say we wanted to construct a 2D random variable whose values lie on a ring. If z is 2D and normally distributed,$g(z) =\frac{z}{10}+ \frac{z}{||z||}$is roughly ring-shaped, as shown in Figure below:
 <img src=\"Normal_Sampling.png" width="450"/>

This is the strategy that VAE and GAN use to create arbitrary distributions: the deterministic function g(generator) is learned from data

# Variational Autoencoder

## Critical Idea
 1. VAE is a explicit density model

Explicit density model means a definition of an explicit density function $p_{model}(x(i);\theta)$, these kind of models' optimization can be straightforward, by taking the gradient uphill algorithm ,by NFL(No Free Lunch) Principle, we find explicit density models with tractable function have some problems. 

To avoid the problems of explicit density models with tractable function, intractable function is introduced to be the density function. Meanwhile, special optimization like variational approximations is used to treat the problem.

 2. VAE is a variant of Autoencoder architecture
 
Unlike classical (sparse, denoising, etc.) autoencoders, Variational autoencoders (VAEs) are generative models, like Generative Adversarial Networks. Their association with this group of models derives mainly from the architectural affinity with the basic autoencoder (the final training objective has an encoder and a decoder), but their mathematical formulation differs significantly. VAEs are directed probabilistic graphical models (DPGM) whose posterior is approximated by a neural network, forming an autoencoder-like architecture.

In other words, VAE has a architecture of Autoencoder, which set a latent space and try to minimize the distance between input and output, but unlike the other Autoencoders, it use probalistic method to treat latent space instead of setting a static latent space which is remained to be mined by dimension reduction map

## VAE Model

1.Treating latent vector as a distribution, Making some notation:
- Prior $p_{\theta}(z)$
- Likelihood $p_{\theta}(x|z)$
- Posterior $p_{\theta}(z|x)$

Addtionally, say we have  a vector  of latent variables in a high-dimensional space Z which we can easily sample according to some probability density function (PDF) P(z) defined over Z

Assuming that we know the real parameter $\theta^*$for this distribution. In order to generate a sample that looks like a real data point $x^{(i)}$, we follow these steps:
1. sample a $z^{(i)}$ from a prior distribution $p_{\theta^*}(\mathbf{z})$.
2. value $x^{(i)}$ is generated from a conditional distribution $p_{\theta^*}(\mathbf{x} \vert \mathbf{z} = \mathbf{z}^{(i)})$

<img src= "Probalistic_Graph.png" width="450"/>



Intuitionally, x is usually image, and z, latent variables for x. Take Mnist as example, x is pixel image and z is things likes angles, strokes thickness etc



but actually we must estimate the $\theta^*$ now

Then we get find we have the foumula $p_\theta(\mathbf{x}^{(i)}) = \int p_\theta(\mathbf{x}^{(i)}\vert\mathbf{z}) p_\theta(\mathbf{z}) d\mathbf{z}$ , In VAEs, the choice of $ p_\theta(\mathbf{x}^{(i)}\vert\mathbf{z})$  is often Gaussian, i.e.,P(X|z;$\theta$) =N(X|f(z;$\theta$),$\sigma^2$). That is, it has mean f(z;$\theta$) and covariance equal to the identity matrix I times some scalar $\sigma$ (which is a hyperparameter). This replacement is necessary to formalize the intuition that some z, needs to result in samples that are merely like X. In general, and particularly early in training, our model will not produce outputs that are identical to any particular X




But we don't optimize it by usual treatment like marginlization, since we found the equation expensive to compute since every $z$ in the latent space should be checked and sum them up(though most of them hardly contribute). To narrow down the search space, approxmation function to output $q_\phi(\mathbf{z}\vert\mathbf{x})$ is introduced here:

In practice, for most z, p(X|z) will be nearly zero, and hence contribute almost nothing to our estimate of p(X). And Q(z|x) can take a value of X and give us a distribution over z values that are likely to produce X. And the space of z values that are likely under Q will be much smaller than the space of all z’s that are likely under the prior P(z). Then the shortcut for P(X) is $\mathbb{E}_{\mathbf{z}\sim q}P(X|Z)$




The graphical model involved in Variational Autoencoder. Solid lines denote the generative distribution p_{$\theta$}(.) and dashed lines denote the distribution $q_\phi (\mathbf{z}\vert\mathbf{x})$ to approximate the intractable posterior $p_\theta (\mathbf{z}\vert\mathbf{x})$
 <img src=\"VAE-graphical-model.png" width="450"/>

To make SGD, we use ELBO loss, but we need make the variational lower bound clearly first

<center>$
\begin{aligned}
& D_\text{KL}( q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}\vert\mathbf{x}) ) & \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z} \vert \mathbf{x})} d\mathbf{z} & \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})p_\theta(\mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} d\mathbf{z} & \scriptstyle{\text{; Because }p(z \vert x) = p(z, x) / p(x)} \\
&=\int q_\phi(\mathbf{z} \vert \mathbf{x}) \big( \log p_\theta(\mathbf{x}) + \log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} \big) d\mathbf{z} & \\
&=\log p_\theta(\mathbf{x}) + \int q_\phi(\mathbf{z} \vert \mathbf{x})\log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z}, \mathbf{x})} d\mathbf{z} & \scriptstyle{\text{; Because }\int q(z \vert x) dz = 1}\\
&=\log p_\theta(\mathbf{x}) + \int q_\phi(\mathbf{z} \vert \mathbf{x})\log\frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{x}\vert\mathbf{z})p_\theta(\mathbf{z})} d\mathbf{z} & \scriptstyle{\text{; Because }p(z, x) = p(x \vert z) p(z)} \\
&=\log p_\theta(\mathbf{x}) + \mathbb{E}_{\mathbf{z}\sim q_\phi(\mathbf{z} \vert \mathbf{x})}[\log \frac{q_\phi(\mathbf{z} \vert \mathbf{x})}{p_\theta(\mathbf{z})} - \log p_\theta(\mathbf{x} \vert \mathbf{z})] &\\
&=\log p_\theta(\mathbf{x}) + D_\text{KL}(q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z})) - \mathbb{E}_{\mathbf{z}\sim q_\phi(\mathbf{z}\vert\mathbf{x})}\log p_\theta(\mathbf{x}\vert\mathbf{z}) &
\end{aligned} 
$<center>
 

Then we have $\log p_\theta(\mathbf{x}) - D_\text{KL}( q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}\vert\mathbf{x}) ) = \mathbb{E}_{\mathbf{z}\sim q_\phi(\mathbf{z}\vert\mathbf{x})}\log p_\theta(\mathbf{x}\vert\mathbf{z}) - D_\text{KL}(q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}))$

This equation is the core of the variational autoencoder, the left hand side has the quantity we want to maximize: log P(X) (plus an error term, which makes Q produce’s that can reproduce a given X; this term will become small if Q is high-capacity). The right hand side is something we can optimize via stochastic gradient descent given the right choice of Q. Note that the framework,In particular, the right hand side of Equation has suddenly taken a form which looks like an autoencoder, since Q is “encoding” X into z, and P is “decoding” it to reconstruct X

<center>$
\begin{aligned}
L_\text{VAE}(\theta, \phi) 
&= -\log p_\theta(\mathbf{x}) + D_\text{KL}( q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}\vert\mathbf{x}) )\\
&= - \mathbb{E}_{\mathbf{z} \sim q_\phi(\mathbf{z}\vert\mathbf{x})} \log p_\theta(\mathbf{x}\vert\mathbf{z}) + D_\text{KL}( q_\phi(\mathbf{z}\vert\mathbf{x}) \| p_\theta(\mathbf{z}) ) \\
\theta^{*}, \phi^{*} &= \arg\min_{\theta, \phi} L_\text{VAE}
\end{aligned}
$<center>

 <img src=\"Learning_process_of_vae.png" width="600"/>

## Pro & Con

### Pros:
 - Principled approach to generative models, benefiting from the explicit model, VAE have a sign function for checking the training progress
 - The decoding map $p(z|x)$ can used for other task such as dimension reduction visualization
 <img src=\"mnist_manifold.png" width="300"/>
 
### Cons:
- Samples blurrier and lower quality compared to state-of-the-art (GAN series, which will be introduced below)


## Experiment 

For the inference network, we use two convolutional layers followed by a fully-connected layer. In the generative network, we mirror this architecture by using a fully-connected layer followed by three convolution transpose layers (a.k.a. deconvolutional layers in some contexts). Note, it's common practice to avoid using batch normalization when training VAEs, since the additional stochasticity due to using mini-batches may aggravate instability on top of the stochasticity from sampling.

Training
- We start by iterating over the dataset
- During each iteration, we pass the image to the encoder to obtain a set of mean and log-variance parameters of the approximate posterior 
- We then apply the reparameterization trick to sample from 
- Finally, we pass the reparameterized samples to the decoder to obtain the logits of the generative distribution 
- Note: Since we use the dataset loaded by keras with 60k datapoints in the training set and 10k datapoints in the test set, our resulting ELBO on the test set is slightly higher than reported results in the literature which uses dynamic binarization of Larochelle's MNIST.

We train VAE in 360 epochs, and the first 90 generation and last 90 genreation is shown below:

 <img src=\"mnist_VAE_GEN.png" width="900"/>

 <img src=\"mnist_VAE_GEN_28-36.png" width="900"/>


# Generative Adversarial Network

## Critical Idea

1. Nash equilibrium

The basic idea of GANs is to set up a game between two players. One of them is called the generator. The generator creates samples that are intended to comefrom the same distribution as the training data.  The other player is thedis-criminator. The discriminator examines samples to determine whether they are  real  or  fake. The  discriminator  learns  using  traditional  supervised  learn-ing techniques,  dividing inputs into two classes (real or fake).  The generatoris trained to fool the discriminator.   


2. GAN is a implicit density model

These models can be trained without even needing to explicitly define a density functions.  These models instead offer a way to train the model while interacting only indirectly with $p_{model}$, usually by sampling from it. Furthermore, GAN can generate a sample in a single step


## GAN 

Although Gan share the same probablistic model with VAE, by introducing two players: Generator and Discriminator, GAisN does beat VAE in geneator, it seems a ridiculous that the more tractable the density function is, the more direct realationship with the density function, the worse result we get finally?


 <img src=\"GAN_Probalistic_Graph.png" width="400"/>

Back to the relationship between GAN and VAE,  the GAN framework can train some models that the VAE framework  cannot  and  vice  versa,  but  the  two  frameworks  also  have  a  large intersection. The most salient difference is that, if relying on standard backprop, VAEs cannot have discrete variables at the input to the generator, while GANs cannot have discrete variables at the output of the generator


In GAN's training process is most straightforward to describe as a game rather than as an optimization problem. In the scenario,both  players  have  cost  functions  that  are  defined  in  terms  of  both  players’  parameters.   The  discriminator  wishes  to  minimize $J^{(D)}(\theta^{(D)},\theta^{(G)})$ and must  do  so  while  controlling  only $\theta^{(D)}$.   The  generator  wishes  to  minimize $J^{(D)}(\theta^{(D)},\theta^{(G)})$ and  must  do  so  while  controlling  only $\theta^{(G)}$.  Because each player’s cost depends on the other player’s parameters, but each player cannot control the other player’s parameters. A Nash equilibrium is a tuple $(\theta^{(D)},\theta^{(G)})$that is a local minimum of $J^{(D)}$ with respect to $\theta^{(D)}$ and a local minimum of $J^{(D)}$with respect to$\theta^{(D)}$

The generator is simply a differentiable function G.  When z is sampled from some simple prior distribution, $G(z)$ yields a sample  of x drawn from $p_{model}$. 

Then cost function must constructed to act SGD algorithm

### The discriminator’s cost

A standard cross-entropy cost is minimized when training a standard binary classifier with a sigmoid output.  The only difference is that the classifier is trained on two minibatches of data; one coming from the dataset, where the label is 1 for all examples, and one coming from the generator, wherethe label is 0 for all examples

<center>$J^{(D)}\left(\boldsymbol{\theta}^{(D)}, \boldsymbol{\theta}^{(G)}\right)=-\frac{1}{2} \mathbb{E}_{\boldsymbol{x} \sim p_{\text {data }}} \log D(\boldsymbol{x})-\frac{1}{2} \mathbb{E}_{\boldsymbol{z}} \log (1-D(G(z)))$<center>
    
More specifically, we want to make sure the discriminator D’s decisions over real data are accurate by maximizing $\mathbb{E}_{x \sim p_{r}(x)} [\log D(x)]$. Meanwhile, given a fake sample $G(z), z \sim p_z(z)$, the discriminator is expected to output a probability, D(G(z)), close to zero by maximizing $\mathbb{E}_{z \sim p_{z}(z)} [\log (1 - D(G(z)))]$

Then the cost function for G should be specified. But let's introuduce zero_sum game, the simplest version of the game is a zero-sum game, in which the sum of all player’s costs is always zero.  In this version of the game $J^{G} = -J^{D}$ , Because$J^{G}$is  tied  directly  to$J^{D}$,  we  can  summarize  the  entire  game with a value function specifying the discriminator’s payoff: 
<center>$V(\theta^{(D)},\theta^{(G)}) = J^{(D)}(\theta^{(D)},\theta^{(G)})$<center>
    
Zero-sum games are also calledminimaxgames because their solution in-volves minimization in an outer loop and maximization in an inner loop:
    
<center>$\boldsymbol{\theta}^{(G) *}=\underset{\boldsymbol{\theta}(G)}{\arg \min } \max _{\boldsymbol{\theta}^{(D)}} V\left(\boldsymbol{\theta}^{(D)}, \boldsymbol{\theta}^{(G)}\right)$<center>
    


 <img src=\"GAN_summary.png" width="400"/>

Now that we have a well-defined loss function. Let’s first examine what is the best value for D.
<center>$L(G, D) = \int_x \bigg( p_{r}(x) \log(D(x)) + p_g (x) \log(1 - D(x)) \bigg) dx$<center>
    
Since we are interested in what is the best value of D(x) to maximize L(G,D), label 
$\tilde{x} = D(x), 
A=p_{r}(x), 
B=p_g(x)$

And then what is inside the integral (we can safely ignore the integral because x is sampled over all the possible values) is:

<center>$
\begin{aligned}
f(\tilde{x}) &=A \log \tilde{x}+B \log (1-\tilde{x}) \\
\frac{d f(\tilde{x})}{d \tilde{x}} &=A \frac{1}{\ln 10} \frac{1}{\tilde{x}}-B \frac{1}{\ln 10} \frac{1}{1-\tilde{x}} \\
&=\frac{1}{\ln 10}\left(\frac{A}{\tilde{x}}-\frac{B}{1-\tilde{x}}\right) \\
&=\frac{1}{\ln 10} \frac{A-(A+B) \tilde{x}}{\tilde{x}(1-\tilde{x})}
\end{aligned}
$<center>

    
Thus, we get the best value of the discriminator: $D^*(x) = \tilde{x}^* = \frac{A}{A + B} = \frac{p_{r}(x)}{p_{r}(x) + p_g(x)} \in [0, 1]$

Once the generator is trained to its optimal, $ p_g $ gets very close to $ p_r $. When$ p_g = p_{r}, D^*(x)$ becomes 1/2

## Pro & Con

Since GANs is the state of the art, we just skip the pros, entering cons section, gans' cons have been well-reserached

1. GAN is hard to train, which means hard to achieve Nash equilibrium
Let’s check out a simple example to better understand why it is difficult to find a Nash equilibrium in an non-cooperative game. Suppose one player takes control of x to minimize $f_1(x)=xy$, while at the same time the other player constantly updates y to minimize $f_2(x)=-xy$.

  Because $\frac{\partial f_1}{\partial x} = y$ and $\frac{\partial f_2}{\partial y} = -x$,  Once x and y have different signs, every following gradient update causes huge oscillation and the instability gets worse in time, as shown in image below 

 <img src=\"nash_equilibrium.png" width="600"/>

2. Vanishing gradient
 
 When the discriminator is perfect, we are guaranteed with $D(x) = 1, \forall x \in p_r $and $D(x) = 0, \forall x \in p_g$. Therefore the loss function L falls to zero and we end up with no gradient to update the loss during learning   iterations

  In fig below, A DCGAN is trained for 1, 10 and 25 epochs. Then, with the generator fixed, a discriminator is trained from scratch and measure the gradients with the original cost function. We see the gradient norms decay quickly 

 <img src=\"GAN_vanishing_gradient.png" width="600"/>

3. Low dimensional supports

   Actually, it's the explaination for the vanishing gradient common in GAN's training
   Following the manifold distribution law, the real world data $p_{data}$concentrate in a lower dimensional manifold,$p_{g}$ lies in a low dimensional manifolds, too. Whenever the generator is asked to a much larger image like 64x64 given a small dimension, such as 100, noise variable input z, the distribution of colors over these 4096 pixels has been defined by the small 100-dimension random number vector and can hardly fill up the whole high dimensional space.

4. Evaluation of generative models

   It's not clear how  to  quantitatively  evaluate  generative  models.   Models  that  obtain  goodlikelihood can generate bad samples,  and models that generate good samplescan  have  poor  likelihood.   There  is  no  clearly  justified  way  to  quantitatively score samples.  GANs are somewhat harder to evaluate than other generative models because it can be difficult to estimate the likelihood for GAN

## Experiment
You use the MNIST dataset to train the generator and the discriminator. The generator will generate handwritten digits resembling the MNIST data.The generator uses tf.keras.layers.Conv2DTranspose (upsampling) layers to produce an image from a seed (random noise). Start with a Dense layer that takes this seed as input, then upsample several times until you reach the desired image size of 28x28x1. Notice the tf.keras.layers.LeakyReLU activation for each layer, except the output layer which uses tanh.The discriminator is a CNN-based image classifier.

We train VAE in 360 epochs, and the first 90 generation and last 90 genreation is shown below:

 <img src=\"MNIST_GAN_GEN_1-9.png" width="900"/>

 <img src=\"MNIST_GAN_GEN_28-36.png" width="900"/>

# References

[1]Goodfellow, Ian, et al. “ Generative Adversarial Networks." NIPS 2016 

[2]Goodfellow, Ian, et al. “Generative adversarial nets.” NIPS, 2014.

[3]Martin Arjovsky and Léon Bottou. “Towards principled methods for training generative adversarial networks.” arXiv preprint arXiv:1701.04862 (2017).

[4]https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html

[5]https://lilianweng.github.io/lil-log/2017/08/20/from-GAN-to-WGAN.html#kullbackleibler-and-jensenshannon-divergence

[6]https://en.wikipedia.org/wiki/Autoencoder

[7]slides from Fei-Dei Li, Justin Johnson, Serena Yeung, cs231n Stanford]

[8]https://blog.evjang.com/2016/08/variational-bayes.html

[9]CARL DOERSCH "Tutorial on Variational Autoencoders" arXiv preprint